Using the DATA_GOV_API_KEY, retrieve data from `https://data.sfgov.org/resource/wg3w-h783`, SF Police Department Report Data.

complete store_to_gcs() to store retrieved data to GCS, and retrieve_data_from_gcs() to retrieve data from GCS.
- store_to_gcs(service_account_key, project_id, bucket_name, file_name, data) : upload data as file_name on the given gcs.
- retrieve_data_from_gcs(service_account_key, project_id, bucket_name, file_name, key_list) : return a list of list which includes values corresponding to keys in key_list.
If 
```
key_list = ['incident_datetime', 'report_datetime', 'incident_code',
            'incident_category', 'incident_description', 'latitude',
            'longitude', 'police_district']
```
This should return
```
[['2025-08-26T23:17:00.000',
  '2025-08-26T23:17:00.000',
  '07041',
  'Recovered Vehicle',
  'Vehicle, Recovered, Auto',
  None,
  None,
  'Out of SF'],
 ['2025-08-27T00:37:00.000',
  '2025-08-27T00:37:00.000',
  '04134',
  'Assault',
  'Battery',
  '37.78041458129883',
  '-122.44901275634766',
  'Park'],....]
```
If a key doesn't exist, its value should be `None`

In [10]:
import datetime
import json
import os 

from dotenv import load_dotenv
from google.oauth2 import service_account
from google.cloud import storage
import requests

In [11]:
load_dotenv()

api_key = os.getenv("DATA_GOV_API_KEY")

- Typically authorization in a header uses the format of..
```
headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
```
However, based on the [documentaiton](https://api.data.gov/docs/developer-manual/), I followed the following format.

Also, the data specific meta data and details can be searchable. (Ex.[link](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783/about_data))

In [12]:
url = 'https://data.sfgov.org/resource/wg3w-h783'
header = {'X-Api-Key':api_key} 

response = requests.get(url, headers=header)

In [13]:
data = response.json()

Create store_to_gcs(service_account_key, project_id, bucket_name, file_name, data) which upload data as file_name on the given gcs.

In [14]:
def store_to_gcs(service_account_key: str,
                 project_id: str,
                 bucket_name: str,
                 file_name: str,
                 data: str) -> None:    
    credentials = service_account.Credentials.from_service_account_file(
        service_account_key
    )
    client = storage.Client(credentials=credentials, project=project_id)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_string(data)

In [15]:
service_account_key = os.getenv("GCP_SERVICE_ACCOUNT_KEY")
project_id = os.getenv("GCP_PROJECT_ID")
bucket_name = os.getenv("GCP_BUCKET_NAME")
file_name = f"sf_police_report/{datetime.date.today()}.json"
store_to_gcs(service_account_key,
            project_id,
            bucket_name,
            file_name,
            json.dumps(data, indent=4))

Complete retrieve_data_from_gcs() to return a list of list which includes values corresponding to keys in key_list.
If 
```
key_list = ['incident_datetime', 'report_datetime', 'incident_code',
            'incident_category', 'incident_description', 'latitude',
            'longitude', 'police_district']
```
This should return
```
[['2025-08-26T23:17:00.000',
  '2025-08-26T23:17:00.000',
  '07041',
  'Recovered Vehicle',
  'Vehicle, Recovered, Auto',
  None,
  None,
  'Out of SF'],
 ['2025-08-27T00:37:00.000',
  '2025-08-27T00:37:00.000',
  '04134',
  'Assault',
  'Battery',
  '37.78041458129883',
  '-122.44901275634766',
  'Park'],....]
```
If a key doesn't exist, its value should be `None`

In [16]:
def retrieve_data_from_gcs(service_account_key: str,
                           project_id: str,
                           bucket_name: str,
                           file_name: str,
                           key_list: list
                           ) -> list:
    credentials = service_account.Credentials.from_service_account_file(
        service_account_key
    )
    client = storage.Client(credentials=credentials, project=project_id)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    data = json.loads(blob.download_as_text())
    filtered_data = [
        {key: record.get(key, None) for key in key_list} for record in data
        ]
    return filtered_data

In [17]:
key_list = ['incident_datetime', 'report_datetime', 'incident_code',
            'incident_category', 'incident_description', 'latitude',
            'longitude', 'police_district']
data = retrieve_data_from_gcs(service_account_key, 
                              project_id,
                              bucket_name,
                              f"sf_police_report/{datetime.date.today()}.json",
                              key_list)

In [18]:
data[:6]

[{'incident_datetime': '2025-08-26T23:17:00.000',
  'report_datetime': '2025-08-26T23:17:00.000',
  'incident_code': '07041',
  'incident_category': 'Recovered Vehicle',
  'incident_description': 'Vehicle, Recovered, Auto',
  'latitude': None,
  'longitude': None,
  'police_district': 'Out of SF'},
 {'incident_datetime': '2025-08-27T00:37:00.000',
  'report_datetime': '2025-08-27T00:37:00.000',
  'incident_code': '04134',
  'incident_category': 'Assault',
  'incident_description': 'Battery',
  'latitude': '37.78041458129883',
  'longitude': '-122.44901275634766',
  'police_district': 'Park'},
 {'incident_datetime': '2025-07-17T15:00:00.000',
  'report_datetime': '2025-08-27T11:55:00.000',
  'incident_code': '09027',
  'incident_category': 'Fraud',
  'incident_description': 'False Personation',
  'latitude': '37.775177001953125',
  'longitude': '-122.45135498046875',
  'police_district': 'Park'},
 {'incident_datetime': '2025-08-23T21:30:00.000',
  'report_datetime': '2025-08-24T14:53:00